# Examples of using velociraptor data

In [ ]:
#load matplotlib as inline so figures display within the notebook
%matplotlib inline
from matplotlib.pylab import *

In [ ]:
#load other useful packages
import numpy as np
import astropy as ap
import pynbody as pyn
#for general python stuff
import sys,os,string,time,re,struct
import math,operator
#for useful scipy stuff
from scipy.stats.mstats import mquantiles
from scipy.misc import comb
import scipy.interpolate as scipyinterp
import scipy.integrate as scipyint
import scipy.optimize as scipyopt
import scipy.special as scipysp
import itertools
#for useful mathematical tools
from sklearn.neighbors import NearestNeighbors
import scipy.spatial as spatial

#to load specific functions defined in another python file
basecodedir='/path/to/dir/'
sys.path.append(basecodedir)
import velociraptor_python_tools as vpt

In [ ]:
#set some plotting parameters
rcParams['figure.figsize'] = (10,6)
rcParams['font.size'] = 18
rc('text', usetex=True)
rcParams['mathtext.fontset'] = 'custom'
rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'
rcParams['xtick.direction']='in'
rcParams['ytick.direction']='in'

## Loading Halos

In [ ]:
#define formats
ASCIIFORMAT=0
HDFFORMAT=2

#base filename 
inputfname='basevelociraptorfilename'

In [ ]:
#now we try loading halo property data
halopropdata,numhalos=vpt.ReadPropertyFile(inputfname,HDFFORMAT)

In [ ]:
#then load the particles associated with halos
haloparticledata=vpt.ReadParticleDataFile(inputfname,HDFFORMAT)

In [ ]:
#list the keys available
print(halopropdata.keys())
print(halopropdata['SimulationInfo'].keys())
print(halopropdata['UnitInfo'].keys())
print(haloparticledata.keys())

## Plotting halo mass function

In [ ]:
#Set bin edges (log M)
NBINS=50
xlim=[7,15]
#xlim=[0,5]
deltax=(xlim[1]-xlim[0])/float(NBINS)
xedges=np.arange(xlim[0],xlim[1]+0.5*deltax,deltax)
xbins=np.arange(xlim[0]+0.5*deltax,xlim[1],deltax)

#desired mass field
massfield=['Mass_200crit','Mass_tot', 'Mass_200mean']
massbindata=dict()
#make histograms while converting units to Msun and kpc
for key in massfield:
    massfac=np.log10(halopropdata['UnitInfo']['Mass_unit_to_solarmass'])
    massbindata[key],blah=np.histogram(np.log10(halopropdata[key])+massfac,xedges)
    massbindata[key]=np.float64(massbindata[key])/np.power(halopropdata['SimulationInfo']['Period']/halopropdata['SimulationInfo']['h_val'],3.0)
#store mass value at which a halo is composed of 100 particles
vlinedata=halopropdata["Mass_tot"][np.where(halopropdata["npart"]==100)][0]

In [ ]:
colors=['red','blue','green']
vlabels=[r'$M_{200\rho_c}$',r'$M_{\rm{tot}}$',r'$M_{200\rho_m}$']
for i in range(len(massfield)):
    plot(np.power(10.0,xbins),massbindata[massfield[i]],color=colors[i],label=vlabels[i],linewidth=2)
axvline(x=vlinedata,color='black',linewidth=2,linestyle='dashed')
ylabel(r'$dn/d\log M$ (kpc$^{-3}$)')
xlabel(r'$M$ [M$_\odot$]')
yscale("log")
xscale("log")
legend()

## Plotting halo versus concentration

In [ ]:
#desired mass field
massfield=['Mass_200crit','Mass_tot', 'Mass_200mean']
massdata=dict()
#get data
sel=np.where(halopropdata['npart']>=1000)
for key in massfield:
    massfac=np.log10(halopropdata['UnitInfo']['Mass_unit_to_solarmass'])
    massdata[key]=np.power(10.0,np.log10(halopropdata[key][sel])+massfac)
cnfw=halopropdata['cNFW'][sel]

In [ ]:
markerstyle=['o','s','^']

for i in range(len(massfield)):
    scatter(massdata[massfield[i]],cnfw,color=colors[i],label=vlabels[i],marker=markerstyle[i],alpha=0.5,s=50)
axvline(x=vlinedata,color='black',linewidth=2,linestyle='dashed')
ylabel(r'$c_{\rm NFW}$')
xlabel(r'$M$ [M$_\odot$]')
yscale("log")
xscale("log")
legend()

## Plotting Vmax distribution

In [ ]:
#Set bin edges (log M)
NBINS=50
xlim=[0.1,3.5]
deltax=(xlim[1]-xlim[0])/float(NBINS)
xedges=np.arange(xlim[0],xlim[1]+0.5*deltax,deltax)
xbins=np.arange(xlim[0]+0.5*deltax,xlim[1],deltax)

#desired mass field
fields=['Vmax','sigV']
bindata=dict()
for key in fields:
    fac=np.log10(halopropdata['UnitInfo']['Velocity_unit_to_kms'])
    bindata[key],blah=np.histogram(np.log10(halopropdata[key])+fac,xedges)
    bindata[key]=np.float64(bindata[key])/np.power(halopropdata['SimulationInfo']['Period']/halopropdata['SimulationInfo']['h_val'],3.0)
#store velocity value at which a halo is composed of 100 particles
vlinedata=np.average(halopropdata["Vmax"][np.where(halopropdata["npart"]==100)])

In [ ]:
colors=['red','blue']
vlabels=[r'$V_{\rm max}$',r'$\sigma_{v}$']
fig1, ax = plt.subplots(figsize=(10,6))
for i in range(len(fields)):
    ax.plot(np.power(10.0,xbins),bindata[fields[i]],color=colors[i],label=vlabels[i],linewidth=2)
ax.axvline(x=vlinedata,color='black',linewidth=2,linestyle='dashed')
ylabel(r'$dn/d\log V$ (kpc$^{-3}$)')
xlabel(r'$V$ [km/s]')
yscale("log")
xscale("log")
ax.legend()

## Plotting Vmax mass distribution

In [ ]:
#desired mass field
massfield=['Mass_200crit','Mass_tot', 'Mass_200mean']
massdata=dict()
sel=np.where(halopropdata['npart']>=20)
massfac=np.log10(halopropdata['UnitInfo']['Mass_unit_to_solarmass'])
for key in massfield:
    massdata[key]=np.power(10.0,np.log10(halopropdata[key][sel])+massfac)
vmax=halopropdata['Vmax'][sel]*halopropdata['UnitInfo']['Velocity_unit_to_kms']
#store mass value at which a halo is composed of 100 particles
massfac=halopropdata['UnitInfo']['Mass_unit_to_solarmass']
vlinedata=halopropdata["Mass_tot"][np.where(halopropdata["npart"]==100)][0]*massfac
#now get median and scatter in relation
NBINS=50
xlim=[7,15]
deltax=(xlim[1]-xlim[0])/float(NBINS)
xedges=np.arange(xlim[0],xlim[1]+0.5*deltax,deltax)
xbins=np.power(10.0,np.arange(xlim[0]+0.5*deltax,xlim[1],deltax))
xedges=np.power(10.0,xedges)
nbindata=dict()
bindata=dict()
for key in massfield:
    nbindata[key]=np.zeros(NBINS)
    bindata[key]=np.zeros([NBINS,6])
    for ibin in range(NBINS):
        wdata=np.where((massdata[key]>=xedges[ibin])*(massdata[key]<xedges[ibin+1]))[0]
        if (wdata.size==0): 
            continue
        nbindata[key][ibin] = wdata.size
        bindata[key][ibin][1:] = np.percentile(vmax[wdata],[2.5,16.0,50.0,84.0,97.5])
        bindata[key][ibin][0]=xbins[ibin]
    wdata=np.where(nbindata[key]>0)[0]
    bindata[key]=bindata[key][wdata].transpose()

In [ ]:
colors=['red','blue','green']
vlabels=[r'$M_{200\rho_c}$',r'$M_{\rm{tot}}$',r'$M_{200\rho_m}$']
fig1, ax = plt.subplots(figsize=(10,6))
for i in range(len(massfield)):
    print(massfield[i])
    ax.plot(bindata[massfield[i]][0],bindata[massfield[i]][3],color=colors[i],label=vlabels[i],linewidth=2,zorder=2)
    ax.fill_between(bindata[massfield[i]][0],bindata[massfield[i]][2],bindata[massfield[i]][4],
                facecolor=colors[i],alpha=0.35,interpolate=True,zorder=1)
    ax.fill_between(bindata[massfield[i]][0],bindata[massfield[i]][1],bindata[massfield[i]][5],
                facecolor=colors[i],alpha=0.15,interpolate=True,zorder=1)
ax.axvline(x=vlinedata,color='black',linewidth=2,linestyle='dashed')
xlabel(r'$M$ [M$_\odot$]')
ylabel(r'$V$ [km/s]')
yscale("log")
xscale("log")
ax.legend()